In [ ]:
pip install wildlife-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 1.5 MB/s eta 0:00:00


In [ ]:
pip install wildlife-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 48.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (5

In [ ]:
pip install timm

**Importing Wildlife data set and Wildlife tools**



In [ ]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

**Same Data set use for Fine tuning**

**MacaqueFaces Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata_MacaqueFaces = datasets.MacaqueFaces('../data/MacaqueFaces')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df, metadata_MacaqueFaces.root, transform=transform)

DATASET MacaqueFaces: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[100:,:], metadata_MacaqueFaces.root, transform=transform)
dataset_query_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[:100,:], metadata_MacaqueFaces.root, transform=transform)

**Importing the pretrain Model**

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_MacaqueFaces = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_MacaqueFaces, database_MacaqueFaces = extractor_MacaqueFaces(dataset_query_MacaqueFaces), extractor_MacaqueFaces(dataset_database_MacaqueFaces)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/204M [00:00<?, ?B/s]

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
 94%|███████████████████████████████████████████████████████████▏   | 46/49 [43:23<02:29, 49.91s/it]

In [ ]:
similarity_function = CosineSimilarity()
similarity_MacaqueFaces = similarity_function(query_MacaqueFaces, database_MacaqueFaces)
print(similarity_MacaqueFaces)

{'cosine': array([[0.93894947, 0.97903377, 0.9748248 , ..., 0.06067665, 0.0604975 ,
        0.07648014],
       [0.94975156, 0.9809722 , 0.9829178 , ..., 0.08857712, 0.08899386,
        0.10716249],
       [0.9368558 , 0.97414136, 0.9836564 , ..., 0.08185371, 0.08442868,
        0.10064524],
       ...,
       [0.9431046 , 0.985183  , 0.98194647, ..., 0.08535906, 0.08368647,
        0.10288547],
       [0.92794454, 0.98648065, 0.97138005, ..., 0.06948385, 0.06463583,
        0.08578961],
       [0.958699  , 0.9781159 , 0.9700406 , ..., 0.11579873, 0.11533174,
        0.13037327]], dtype=float32)}


In [ ]:
classifier_MacaqueFaces = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
predictions_MacaqueFaces = classifier_MacaqueFaces(similarity_MacaqueFaces['cosine'])
print("Predictions for 100 test Images:-\n",predictions_MacaqueFaces)
accuracy_MacaqueFaces = np.mean(dataset_query_MacaqueFaces.labels_string == predictions_MacaqueFaces)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_MacaqueFaces * 100))
# precision
precision_MacaqueFaces = precision_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
# recall
recall_MacaqueFaces = recall_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
# F1 score
f1_MacaqueFaces = f1_score(dataset_query_MacaqueFaces.labels_string, predictions_MacaqueFaces, average='weighted')
print("Precision:", precision_MacaqueFaces)
print("Recall:", recall_MacaqueFaces)
print("F1 Score:", f1_MacaqueFaces)

Predictions for 100 test Images:-
 ['Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan']
Accuracy on MacaqueFaces data: 100.00%


**LionData Dateset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata_LionData = datasets.LionData('../data/LionData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_LionData = WildlifeDataset(metadata_LionData.df, metadata_LionData.root, transform=transform)

DATASET LionData: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_LionData = WildlifeDataset(metadata_LionData.df.iloc[100:,:], metadata_LionData.root, transform=transform)
dataset_query_LionData = WildlifeDataset(metadata_LionData.df.iloc[:100,:], metadata_LionData.root, transform=transform)

In [ ]:
name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_LionData = DeepFeatures(timm.create_model(name, num_classes=0, pretrained=True))
query_LionData, database_LionData = extractor_LionData(dataset_query_LionData), extractor_LionData(dataset_database_LionData)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.69s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 5/5 [03:30<00:00, 42.13s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_LionData = similarity_function(query_LionData, database_LionData)
print(similarity_LionData)

{'cosine': array([[0.86896247, 0.69942033, 0.8777389 , ..., 0.829781  , 0.8954884 ,
        0.86451447],
       [0.9500992 , 0.7055931 , 0.9380013 , ..., 0.7477597 , 0.9478767 ,
        0.89982736],
       [0.87242615, 0.6950849 , 0.89943147, ..., 0.75892425, 0.91286826,
        0.9114601 ],
       ...,
       [0.8723602 , 0.7832219 , 0.9014683 , ..., 0.8094394 , 0.9129671 ,
        0.9049461 ],
       [0.8919123 , 0.73857903, 0.92439425, ..., 0.7845442 , 0.9392824 ,
        0.9575528 ],
       [0.89837366, 0.75758326, 0.91506803, ..., 0.80790067, 0.9342222 ,
        0.93150544]], dtype=float32)}


In [ ]:
# Predictions
classifier_LionData = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
predictions_LionData = classifier_LionData(similarity_LionData['cosine'])
print("Predictions for 100 test Images:-\n",predictions_LionData)
# Accuracy
accuracy_LionData = np.mean(dataset_query_LionData.labels_string == predictions_LionData)
print("Accuracy on Lion data: {:.2f}%".format(accuracy_LionData * 100))
# F1 score,recall and precision
precision = precision_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted')
recall = recall_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted')
f1 = f1_score(dataset_query_LionData.labels_string, predictions_LionData, average='weighted')
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Predictions for 100 test Images:-
 ['Sebastian' 'Saba' 'Etana' 'Tigisi' 'Sebastian' 'Moswen' 'Naape'
 'Simaloi' 'Autumn' 'Lolparpit' 'Maskio' 'Chaimu' 'Naape' 'Saba' 'Saitoti'
 'Esiriwua' 'Sadala' 'Etana' 'Enderoni' 'Kinna' 'Lucinda' 'Enkume' 'Joy'
 'Lemuanik' 'Naibor' 'Shambe' 'Siti' 'Selenkay' 'Sadala' 'Manzy' 'Moswen'
 'Napi' 'Ngare' 'Naimina' 'Napi' 'Cleopatra' 'Moswen' 'Kioni' 'Chaimu'
 'Engiyaa' 'Enderoni' 'Jamal' 'Leia' 'Sebastian' 'Nuru' 'Maskio' 'Mooza'
 'Moswen' 'Kibibi' 'Tigisi' 'Enderoni' 'Lucinda' 'Moswen' 'Kioni'
 'Sebastian' 'Mashavu' 'Mickey' 'Mama-Kali' 'Kioni' 'Ngare' 'Nashipai'
 'Sikio-Kali' 'Ngare' 'Amber' 'Olonyori' 'Lemuanik' 'Kibibi' 'Ngare'
 'Saimutie' 'Amber' 'Shemsa' 'Olonyori' 'Mickey' 'Esiriwua' 'Nashipai'
 'Enderoni' 'Summer' 'Nuru' 'Mickey' 'Joy' 'Cleopatra' 'Tigisi' 'Mooza'
 'Saitoti' 'Kioni' 'Naini' 'Rescue' 'Sikio-Kali' 'Empurra' 'Enadalut'
 'Lemayian' 'Rescue' 'Enadalut' 'Esiriwua' 'Enkume' 'Senteu' 'Doto'
 'Enkume' 'Nakato' 'Enderoni']
Accuracy on Lio

**NyalaDataSet**

In [ ]:
# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata_N = datasets.NyalaData('../data/NyalaData')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset_N = WildlifeDataset(metadata_N.df, metadata_N.root, transform=transform)

DATASET NyalaData: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_N = WildlifeDataset(metadata_N.df.iloc[100:,:], metadata_N.root, transform=transform)
dataset_query_N = WildlifeDataset(metadata_N.df.iloc[:100,:], metadata_N.root, transform=transform)

In [ ]:
query_N, database_N = extractor_N(dataset_query_N), extractor_N(dataset_database_N)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 15/15 [10:46<00:00, 43.08s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_N = similarity_function(query_N, database_N)
print(similarity_N)

{'cosine': array([[0.8619583 , 0.01217471, 0.05642469, ..., 0.6895717 , 0.03031028,
        0.898418  ],
       [0.25952083, 0.918558  , 0.785017  , ..., 0.5010886 , 0.82998675,
        0.1797384 ],
       [0.71705055, 0.4680481 , 0.5621629 , ..., 0.962554  , 0.5190879 ,
        0.7367308 ],
       ...,
       [0.2627344 , 0.9283303 , 0.7531287 , ..., 0.4816041 , 0.80183464,
        0.17906114],
       [0.84421116, 0.0514888 , 0.10151841, ..., 0.7681327 , 0.0811858 ,
        0.9342401 ],
       [0.07699895, 0.7878246 , 0.9208322 , ..., 0.38661018, 0.9427093 ,
        0.07355451]], dtype=float32)}


In [ ]:
# Predictions
classifier_N = KnnClassifier(k=1, database_labels=dataset_database_N.labels_string)
predictions_N = classifier_N(similarity_N['cosine'])
print("Predictions for 100 test Images:-\n",predictions_N)
# Accuracy
accuracy_N = np.mean(dataset_query_N.labels_string == predictions_N)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_N * 100))
# precision,recall and F1 score
precision_N = precision_score(dataset_query_N.labels_string, predictions_N, average='weighted')
recall_N = recall_score(dataset_query_N.labels_string, predictions_N, average='weighted')
f1_N = f1_score(dataset_query_N.labels_string, predictions_N, average='weighted')
print("Precision:", precision_N)
print("Recall:", recall_N)
print("F1 Score:", f1_N)

Predictions for 100 test Images:-
 ['166' '10' '57' '89' '58' '179' '55' '58' '149' '71' '87' '125' '56'
 '144' '131' '8' '36' '150' '131' '11' '116' '19' '20' '86' '260' '165'
 '140' '11' '246' '41' '140' '197' '9' '131' '24' '48' '41' '11' '48'
 '131' '272' '47' '21' '46' '8' '241' '114' '62' '21' '48' '252' '151'
 '27' '108' '82' '51' '101' '85' '32' '58' '46' '139' '63' '85' '9' '181'
 '65' '242' '43' '144' '142' '12' '60' '5' '127' '166' '56' '21' '27' '36'
 '20' '188' '51' '51' '42' '20' '111' '62' '165' '68' '104' '174' '48'
 '217' '144' '6' '50' '91' '105' '140']
Accuracy on NyalaData data: 11.00%


**StripeSpotter Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')
# Load dataset metadata
metadata = datasets.StripeSpotter('../data/StripeSpotter')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET StripeSpotter: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.88s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 6/6 [04:25<00:00, 44.30s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[0.2130194 , 0.588757  , 0.65936303, ..., 0.4394258 , 0.44544154,
        0.621335  ],
       [0.17780262, 0.38624054, 0.40102234, ..., 0.7090034 , 0.20244715,
        0.2846234 ],
       [0.6676848 , 0.43850845, 0.34033626, ..., 0.2868997 , 0.39746365,
        0.36011013],
       ...,
       [0.60642976, 0.23701996, 0.21666232, ..., 0.41781956, 0.32831538,
        0.35214955],
       [0.51512223, 0.2798951 , 0.32729602, ..., 0.53868353, 0.39507398,
        0.28750026],
       [0.17924337, 0.24542159, 0.3013066 , ..., 0.58145046, 0.2895743 ,
        0.35858634]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy * 100))
# precision
precision = precision_score(dataset_query.labels_string, predictions, average='weighted')
# recall
recall = recall_score(dataset_query.labels_string, predictions, average='weighted')
# F1 score
f1 = f1_score(dataset_query.labels_string, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Predictions for 100 test Images:-
 ['09_212' 'S09_161' '08_085' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' 'S09_067' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '09_212' 'OP05_625' 'OP05_625' '02_149'
 '01_230' '09_200' 'OP05_625' '01_661' 'OP04_225' '01_661' '01_661'
 'S09_174' '01_679' '01_661' 'S09_067' 'S09_111' 'S09_112' '09_212'
 'OP04_225' '02_161' '09_185' '04_008' '08_100' '01_460' '01_679'
 'S09_111' '01_679' '01_036' 'S09_067']
Accuracy on Str

**IPanda50 Dataset**

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=174954f6-d2bd-4289-b1ed-6c990eb70713
To: /data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:15<00:00, 59.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 3.27MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 21.5MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:38<00:00, 38.09s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 53/53 [38:16<00:00, 43.33s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[ 0.02714637,  0.53501487,  0.14056858, ...,  0.38829625,
         0.43481827, -0.07561179],
       [ 0.08373775,  0.5616951 ,  0.03785922, ...,  0.3602667 ,
         0.6493416 , -0.1295084 ],
       [ 0.01190435,  0.4141855 ,  0.23542283, ...,  0.45754877,
         0.5449476 , -0.1335805 ],
       ...,
       [ 0.0320384 ,  0.12872331, -0.14140168, ...,  0.02448   ,
         0.22118227, -0.17661369],
       [-0.08624166,  0.16572046,  0.07199848, ...,  0.14675027,
         0.23473434, -0.02312863],
       [ 0.01022829,  0.06892081,  0.7597883 , ...,  0.4224525 ,
         0.10611828, -0.04714176]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '37_xinger' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '38_xingfan' '04_chengdui' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '24_qixi' '10_hexing' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '10_hexing'
 '29_shuangxiong' '11_jiaoao' '44_yayi' '14_maodou' '30_shuqing'
 '36_xingda' '39_xinghui' '33_wuyi' '10_hexing' '10_hexing' '00_aibang'
 '49_yuanrun' '35_xilan' '04_chengdui' '10_hexing' '39_xinghui' '44_yayi'
 '10_hexing' '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui'
 '10_hexing' '17_meibang' '00_aibang' '46_yazhu' '14_maodou' '15_maosun'
 '32_susu' '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomia

CZoo Dataset

In [ ]:
# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

DATASET CZoo: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
query, database = extractor(dataset_query), extractor(dataset_database)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|█████████████████████████████████████████████████████████████████| 1/1 [00:35<00:00, 35.28s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 16/16 [10:51<00:00, 40.69s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity = similarity_function(query, database)
print(similarity)

{'cosine': array([[ 9.0549982e-01,  8.9250720e-01,  8.9013600e-01, ...,
        -2.4628341e-02, -1.9136328e-02,  2.0032311e-02],
       [ 9.1913915e-01,  9.1288275e-01,  9.0220594e-01, ...,
        -1.2885875e-02, -3.0940475e-02, -7.5778235e-03],
       [ 9.3326151e-01,  9.2265338e-01,  9.1522133e-01, ...,
        -3.5657752e-02, -1.6588740e-02,  1.8029639e-02],
       ...,
       [ 9.4986129e-01,  9.6489835e-01,  9.4513679e-01, ...,
        -5.2607138e-02, -3.3724420e-03, -1.1571882e-02],
       [ 9.5598102e-01,  9.6259630e-01,  9.4699091e-01, ...,
        -4.5473136e-02, -1.4542699e-02, -3.2175947e-03],
       [ 9.4659024e-01,  9.6087039e-01,  9.4981194e-01, ...,
        -4.7210053e-02, -3.1141117e-03, -8.7950751e-04]], dtype=float32)}


In [ ]:
classifier = KnnClassifier(k=1, database_labels=dataset_database.labels_string)
predictions = classifier(similarity['cosine'])
print("Predictions for 100 test Images:-\n",predictions)
accuracy = np.mean(dataset_query.labels_string == predictions)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy * 100))

Predictions for 100 test Images:-
 ['Alex' 'Alex' 'Alex' 'Jahaga' 'Jahaga' 'Alex' 'Alex' 'Jahaga' 'Alex'
 'Jahaga' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex' 'Alex'
 'Alex' 'Alex' 'Alex' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Gertrudia' 'Gertrudia' 'Gertrudia' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Annett' 'Annett' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra' 'Alexandra'
 'Alexandra' 'Alexandra' 'Al

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Calculate precision
precision = precision_score(dataset_query.labels_string, predictions, average='weighted')

# Calculate recall
recall = recall_score(dataset_query.labels_string, predictions, average='weighted')

# Calculate F1 score
f1 = f1_score(dataset_query.labels_string, predictions, average='weighted')

print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


NameError: name 'dataset_query' is not defined

In [ ]:

# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [02:29, 29.8MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



1485


# ***SIFT***

In [1]:
pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 30.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylin

In [2]:
pip install wildlife-datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 862.3 kB/s eta 0:00:00


In [3]:
pip install wildlife-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.5 MB/s eta 0:00:00


In [4]:
import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
from wildlife_tools.features import SIFTFeatures

In [ ]:

# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)

DATASET CowDataset: DOWNLOADING STARTED.


31210192: 4.45GB [02:41, 27.6MB/s]                            


DATASET CowDataset: EXTRACTING STARTED.
DATASET CowDataset: FINISHED.



In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.CowDataset.get_data('../data/CowDataset')

# Load dataset metadata
metadata_CZoo = datasets.CowDataset('../data/CowDataset')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

DATASET CowDataset: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

100%|███████████████████████████████████████████████████████████| 1385/1385 [06:44<00:00,  3.42it/s]


In [ ]:
print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

First 5 query features shape: [(633, 128), (535, 128), (363, 128), (445, 128), (704, 128)]
First 5 database features shape: [(667, 128), (610, 128), (527, 128), (647, 128), (697, 128)]


In [ ]:
import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

In [ ]:
similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)

100%|██████████████████████████████████████████████████████| 138500/138500 [06:00<00:00, 383.68it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[120.  93.  88. ...  69.  94.  82.]
 [110. 101.  95. ...  76.  98.  93.]
 [107. 101.  87. ...  66.  93.  99.]
 ...
 [126. 108. 101. ...  75. 106.  86.]
 [106.  99.  71. ...  72.  93.  86.]
 [120.  86.  95. ...  81. 109. 128.]]


In [ ]:
classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_CZoo * 100))



Predictions for 100 test Images:-
 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '13' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1'
 '2' '2' '5' '9' '13' '1' '10' '3' '11' '12' '5' '3' '6' '5' '13' '2' '7'
 '6' '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8'
 '10' '1' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']
Accuracy on MacaqueFaces data: 94.00%


In [ ]:
# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")

Prediction 	 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '13' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1'
 '2' '2' '5' '9' '13' '1' '10' '3' '11' '12' '5' '3' '6' '5' '13' '2' '7'
 '6' '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8'
 '10' '1' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']
Ground truth 	 ['2' '9' '2' '13' '1' '6' '4' '7' '1' '4' '10' '9' '13' '3' '6' '10' '6'
 '4' '2' '13' '6' '2' '2' '9' '8' '8' '4' '7' '2' '12' '6' '6' '2' '4' '5'
 '2' '7' '9' '4' '5' '9' '11' '1' '8' '8' '1' '9' '4' '3' '7' '12' '1' '2'
 '5' '5' '9' '4' '1' '10' '3' '11' '12' '5' '3' '6' '5' '5' '2' '7' '11'
 '8' '5' '3' '9' '8' '7' '2' '1' '11' '1' '6' '3' '12' '6' '3' '8' '10'
 '11' '7' '6' '1' '8' '6' '3' '11' '10' '10' '2' '3' '13']

 Accuracy:  94.0 %


In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo = SIFTFeatures()

query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_CZoo * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET CZoo: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


100%|███████████████████████████████████████████████████████████| 2009/2009 [00:55<00:00, 36.42it/s]


First 5 query features shape: [(218, 128), (177, 128), (192, 128), (185, 128), (140, 128)]
First 5 database features shape: [(529, 128), (752, 128), (792, 128), (257, 128), (403, 128)]


100%|█████████████████████████████████████████████████████| 200900/200900 [02:18<00:00, 1454.03it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[ 79. 118.  79. ...  63.  32.  62.]
 [ 85. 108. 108. ...  65.  31.  68.]
 [ 66. 122. 115. ...  77.  44.  62.]
 ...
 [ 59. 125.  98. ...  85.  34.  51.]
 [ 73. 114. 104. ...  50.  42.  58.]
 [ 87. 127. 128. ...  66.  44.  50.]]
Predictions for 100 test Images:-
 ['Alex' 'Jahaga' 'Riet' 'Jahaga' 'Gertrudia' 'Jahaga' 'Fraukje' 'Riet'
 'Alex' 'Alex' 'Riet' 'Riet' 'Alex' 'Fraukje' 'Jahaga' 'Alex' 'Alex'
 'Fraukje' 'Alex' 'Jahaga' 'Alex' 'Jahaga' 'Annett' 'Jahaga' 'Gertrudia'
 'Gertrudia' 'Alex' 'Jahaga' 'Alex' 'Fraukje' 'Fraukje' 'Riet' 'Gertrudia'
 'Riet' 'Gertrudia' 'Alex' 'Fraukje' 'Annett' 'Annett' 'Jahaga' 'Jahaga'
 'Jahaga' 'Alex' 'Riet' 'Alex' 'Jahaga' 'Gertrudia' 'Riet' 'Gertrudia'
 'Jahaga' 'Fraukje' 'Gertrudia' 'Riet' 'Riet' 'Gertrudia' 'Gertrudia'
 'Gertrudia' 'Annett' 'Jahaga' 'Jahaga' 'Annett' 'Riet' 'Fraukje' 'Jahaga'
 'Gertrudia' 'Annett' 'Annett' 'Gertrudia' 'Annett' 'Jahaga' 'Alex'
 'Gertrudia' 'Jahaga' 'Jaha

In [ ]:
# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=389a70d9-7ff1-4463-a2cf-be3751456328
To: /data/IPanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:16<00:00, 56.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /data/IPanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 4.51MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /data/IPanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 21.3MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/Ipanda50')

# Load dataset metadata
metadata_Ipanda50 = datasets.IPanda50('../data/Ipanda50')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_Ipanda50.df, metadata_Ipanda50.root, transform=transform)
dataset_database_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[100:], metadata_Ipanda50.root, transform=transform)
dataset_query_Ipanda50 = WildlifeDataset(metadata_Ipanda50.df.iloc[:100], metadata_Ipanda50.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_Ipanda50 = SIFTFeatures()

query_Ipanda50, database_Ipanda50 = extractor_Ipanda50(dataset_query_Ipanda50), extractor_Ipanda50(dataset_database_Ipanda50)

print(f'First 5 query features shape: {[i.shape for i in query_Ipanda50[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_Ipanda50[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_Ipanda50, database_Ipanda50)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_Ipanda50 = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
predictions_Ipanda50 = classifier_Ipanda50(sim)

print("Predictions for 100 test Images:-\n",predictions_Ipanda50)
accuracy_Ipanda50 = np.mean(dataset_query_Ipanda50.labels_string == predictions_Ipanda50)
print("Accuracy on Ipanda50 data: {:.2f}%".format(accuracy_Ipanda50 * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_Ipanda50.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_Ipanda50.labels_string)

acc = sum(preds == dataset_query_Ipanda50.labels_string) / len(dataset_query_Ipanda50.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET IPanda50: DOWNLOADING STARTED.


Downloading...
From (original): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg
From (redirected): https://drive.google.com/uc?id=1nkh-g6a8JvWy-XsMaZqrN2AXoPlaXuFg&confirm=t&uuid=1c9ceb20-4ebd-40ab-9cc3-6b3f79c4e4ba
To: /data/Ipanda50/iPanda50-images.zip
100%|██████████| 926M/926M [00:07<00:00, 116MB/s]
Downloading...
From: https://drive.google.com/uc?id=1gVREtFWkNec4xwqOyKkpuIQIyWU_Y_Ob
To: /data/Ipanda50/iPanda50-split.zip
100%|██████████| 140k/140k [00:00<00:00, 4.59MB/s]
Downloading...
From: https://drive.google.com/uc?id=1jdACN98uOxedZDT-6X3rpbooLAAUEbNY
To: /data/Ipanda50/iPanda50-eyes-labels.zip
100%|██████████| 2.55M/2.55M [00:00<00:00, 21.7MB/s]


DATASET IPanda50: EXTRACTING STARTED.
DATASET IPanda50: FINISHED.



100%|███████████████████████████████████████████████████████████| 6774/6774 [04:13<00:00, 26.68it/s]


First 5 query features shape: [(179, 128), (320, 128), (431, 128), (410, 128), (272, 128)]
First 5 database features shape: [(302, 128), (171, 128), (533, 128), (312, 128), (159, 128)]


100%|██████████████████████████████████████████████████████| 677400/677400 [12:07<00:00, 931.15it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[ 46.  43.  95. ...  89.  53.  64.]
 [ 62.  34. 103. ...  86.  50.  70.]
 [ 55.  23.  86. ...  80.  41.  72.]
 ...
 [ 45.  32.  91. ...  79.  54.  75.]
 [ 49.  26.  62. ...  85.  40.  63.]
 [ 48.  33.  76. ... 103.  55.  66.]]
Predictions for 100 test Images:-
 ['22_nini' '35_xilan' '35_xilan' '35_xilan' '35_xilan' '22_nini'
 '09_fushun' '35_xilan' '46_yazhu' '22_nini' '08_fulai' '35_xilan'
 '35_xilan' '39_xinghui' '35_xilan' '35_xilan' '14_maodou' '45_yayun'
 '35_xilan' '35_xilan' '35_xilan' '35_xilan' '35_xilan' '00_aibang'
 '35_xilan' '15_maosun' '16_maotao' '00_aibang' '35_xilan' '02_baolan'
 '35_xilan' '22_nini' '35_xilan' '35_xilan' '00_aibang' '35_xilan'
 '35_xilan' '02_baolan' '00_aibang' '26_qiyuan' '35_xilan' '35_xilan'
 '08_fulai' '35_xilan' '35_xilan' '00_aibang' '35_xilan' '35_xilan'
 '14_maodou' '22_nini' '35_xilan' '46_yazhu' '35_xilan' '23_qiubang'
 '35_xilan' '35_xilan' '02_baolan' '35_xilan' '35_xilan' 

Liondata

In [7]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.LionData.get_data('../data/LionData')

# Load dataset metadata
metadata_LionData = datasets.LionData('../data/LionData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_LionData.df, metadata_LionData.root, transform=transform)
dataset_database_LionData = WildlifeDataset(metadata_LionData.df.iloc[100:], metadata_LionData.root, transform=transform)
dataset_query_LionData = WildlifeDataset(metadata_LionData.df.iloc[:100], metadata_LionData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_LionData = SIFTFeatures()

query_LionData, database_LionData = extractor_LionData(dataset_query_LionData), extractor_LionData(dataset_database_LionData)

print(f'First 5 query features shape: {[i.shape for i in query_LionData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_LionData[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.6])
sim = similarity(query_LionData, database_LionData)[0.6]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_LionData = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
predictions_LionData = classifier_LionData(sim)

print("Predictions for 100 test Images:-\n",predictions_LionData)
accuracy_LionData = np.mean(dataset_query_LionData.labels_string == predictions_LionData)
print("Accuracy on LionData data: {:.2f}%".format(accuracy_LionData * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_LionData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_LionData.labels_string)

acc = sum(preds == dataset_query_LionData.labels_string) / len(dataset_query_LionData.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET LionData: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


100%|█████████████████████████████████████████████████████████████| 640/640 [00:13<00:00, 46.12it/s]


First 5 query features shape: [(262, 128), (132, 128), (285, 128), (236, 128), (135, 128)]
First 5 database features shape: [(297, 128), (217, 128), (175, 128), (173, 128), (160, 128)]


100%|███████████████████████████████████████████████████████| 64000/64000 [00:39<00:00, 1625.03it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[1. 1. 2. ... 2. 2. 2.]
 [4. 3. 0. ... 1. 3. 4.]
 [5. 4. 1. ... 2. 2. 3.]
 ...
 [5. 5. 1. ... 3. 2. 5.]
 [5. 2. 4. ... 2. 5. 7.]
 [1. 7. 1. ... 5. 6. 2.]]
Predictions for 100 test Images:-
 ['Saruni' 'Arria' 'Amber' 'Opi' 'Jamal' 'Moswen' 'Empurra' 'Enkume' 'Supu'
 'Yieyio' 'Mama-Kali' 'Sangiki' 'Sero' 'Chaimu' 'Simaloi' 'Lolparpit'
 'Chaimu' 'Naape' 'Empurra' 'Opi' 'Romeo' 'Chaimu' 'Naini' 'Nguro'
 'Esiriwua' 'Nguro' 'Ayo' 'Enderoni' 'Spring' 'Sero' 'Opi' 'Enadalut'
 'Bibi' 'Tikki' 'Chaimu' 'Opi' 'Dada' 'Saruni' 'Safi' 'Saruni' 'Spring'
 'Samir' 'Lolparpit' 'Saba' 'Kinna' 'Senteu' 'Namunyak' 'Sero' 'Naibor'
 'Naini' 'Opi' 'Ayo' 'Nariku-Inkgera' 'Olonyori' 'Ngare' 'Lemayian'
 'Chaimu' 'Sangiki' 'Dada' 'Naini' 'Moswen' 'Tikki' 'Joshua' 'Tikki'
 'Spring' 'Siti' 'Nakepari' 'Moswen' 'Kaka' 'Chaimu' 'Spot' 'Opi' 'Nuru'
 'Esiriwua' 'Opi' 'Nashipai' 'Winter' 'Esiriwua' 'Sangiki'
 'Nariku-Inkgera' 'Lilly' 'Yieyio' 'Nariku-Inkger


MacaqueFaces



In [10]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.MacaqueFaces.get_data('../data/MacaqueFaces')

# Load dataset metadata
metadata_MacaqueFaces = datasets.MacaqueFaces('../data/MacaqueFaces')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_MacaqueFaces.df, metadata_MacaqueFaces.root, transform=transform)

dataset_database_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[100:], metadata_MacaqueFaces.root, transform=transform)
dataset_query_MacaqueFaces = WildlifeDataset(metadata_MacaqueFaces.df.iloc[:100], metadata_MacaqueFaces.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_MacaqueFaces = SIFTFeatures()

query_MacaqueFaces, database_MacaqueFaces = extractor_MacaqueFaces(dataset_query_MacaqueFaces), extractor_MacaqueFaces(dataset_database_MacaqueFaces)



DATASET MacaqueFaces: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


100%|███████████████████████████████████████████████████████████| 6180/6180 [01:20<00:00, 76.90it/s]


In [11]:
print(f'First 5 query features shape: {[i.shape for i in query_MacaqueFaces[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_MacaqueFaces[:5]]}')

First 5 query features shape: [(24, 128), (37, 128), (23, 128), (38, 128), (20, 128)]


AttributeError: 'NoneType' object has no attribute 'shape'

In [14]:
import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_MacaqueFaces, database_MacaqueFaces)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_MacaqueFaces = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
predictions_MacaqueFaces = classifier_MacaqueFaces(sim)

print("Predictions for 100 test Images:-\n",predictions_MacaqueFaces)
accuracy_MacaqueFaces = np.mean(dataset_query_MacaqueFaces.labels_string == predictions_MacaqueFaces)
print("Accuracy on MacaqueFaces data: {:.2f}%".format(accuracy_MacaqueFaces * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_MacaqueFaces.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_MacaqueFaces.labels_string)

acc = sum(preds == dataset_query_MacaqueFaces.labels_string) / len(dataset_query_MacaqueFaces.labels_string)
print('\n Accuracy: ', acc*100,"%")


100%|████████████████████████████████████████████████████| 618000/618000 [00:41<00:00, 14990.50it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[ 0.  2.  6. ...  0.  3.  1.]
 [ 0.  1.  2. ...  1.  1.  0.]
 [ 0.  4.  4. ...  0.  2.  0.]
 ...
 [ 0.  9.  4. ...  0.  2.  1.]
 [ 0.  4. 11. ...  1.  0.  0.]
 [ 0.  6.  9. ...  1.  1.  0.]]
Predictions for 100 test Images:-
 ['Thyme' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Tamara' 'Saphy' 'Dan'
 'Dan' 'Dan' 'Saphy' 'Dan' 'Teal' 'Saphy' 'Dan' 'Dan' 'Dan' 'Sonja' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Saphy' 'Serena'
 'Rupee' 'Dan' 'Dan' 'Rupee' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Tamara' 'Tea' 'Judd' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Rupee' 'Lydia' 'Dan' 'Dan'
 'Dan' 'Dan' 'Dan' 'Dan' 'Dan' 'Saphy' 'Dan' 'Thyme' 'Dan' 'Meg' 'Rupee'
 'Rupee' 'Judd' 'Dan' 'Rupee' 'Dan' 'Dan' 'Serena' 'Dan' 'Dan' 'Dan'
 'Judd' 'Dan' 'Serena' 'Judd' 'Serena' 'Serena' 'Dan' 'Tamara' 'Dan'
 'Saphy' 'Serena' 'Judd' 'Dan']
Accuracy on MacaqueFaces data

NyalaDataSet

In [5]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.NyalaData.get_data('../data/NyalaData')

# Load dataset metadata
metadata_NyalaData = datasets.NyalaData('../data/NyalaData')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_NyalaData.df, metadata_NyalaData.root, transform=transform)
dataset_database_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[100:], metadata_NyalaData.root, transform=transform)
dataset_query_NyalaData = WildlifeDataset(metadata_NyalaData.df.iloc[:100], metadata_NyalaData.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_NyalaData = SIFTFeatures()

query_NyalaData, database_NyalaData = extractor_NyalaData(dataset_query_NyalaData), extractor_NyalaData(dataset_database_NyalaData)

print(f'First 5 query features shape: {[i.shape for i in query_NyalaData[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_NyalaData[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_NyalaData, database_NyalaData)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_NyalaData = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
predictions_NyalaData = classifier_NyalaData(sim)

print("Predictions for 100 test Images:-\n",predictions_NyalaData)
accuracy_NyalaData = np.mean(dataset_query_NyalaData.labels_string == predictions_NyalaData)
print("Accuracy on NyalaData data: {:.2f}%".format(accuracy_NyalaData * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_NyalaData.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_NyalaData.labels_string)

acc = sum(preds == dataset_query_NyalaData.labels_string) / len(dataset_query_NyalaData.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET NyalaData: DOWNLOADING STARTED.


main.zip: 495MB [00:19, 25.5MB/s]


DATASET NyalaData: EXTRACTING STARTED.
DATASET NyalaData: FINISHED.



100%|███████████████████████████████████████████████████████████| 1842/1842 [02:53<00:00, 10.59it/s]


First 5 query features shape: [(738, 128), (857, 128), (720, 128), (756, 128), (621, 128)]
First 5 database features shape: [(727, 128), (829, 128), (679, 128), (613, 128), (651, 128)]


100%|██████████████████████████████████████████████████████| 184200/184200 [13:36<00:00, 225.57it/s]


Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[20. 34. 37. ... 21. 35. 26.]
 [26. 28. 24. ... 23. 25. 22.]
 [22. 38. 45. ... 27. 38. 25.]
 ...
 [20. 27. 26. ... 14. 30. 21.]
 [27. 39. 32. ... 17. 34. 28.]
 [34. 33. 41. ... 30. 40. 15.]]
Predictions for 100 test Images:-
 ['30' '24' '22' '22' '79' '155' '149' '193' '21' '30' '140' '22' '34'
 '164' '30' '21' '30' '120' '30' '246' '87' '34' '140' '43' '140' '21'
 '89' '246' '239' '30' '246' '246' '34' '140' '23' '140' '120' '140' '209'
 '140' '272' '30' '24' '30' '51' '8' '21' '25' '140' '267' '34' '122' '30'
 '140' '22' '140' '246' '246' '120' '34' '50' '140' '140' '79' '74' '246'
 '24' '243' '10' '140' '246' '246' '246' '30' '246' '20' '34' '14' '125'
 '30' '246' '188' '246' '172' '120' '140' '34' '71' '34' '246' '158' '82'
 '33' '55' '94' '196' '190' '91' '31' '34']
Accuracy on NyalaData data: 5.00%
Prediction 	 ['30' '24' '22' '22' '79' '155' '149' '193' '21' '30' '140' '22' '34'
 '164' '30' '21' '30' '120' '30' '2

StripeSpotter Dataset

In [4]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.StripeSpotter.get_data('../data/StripeSpotter')

# Load dataset metadata
metadata_StripeSpotter = datasets.StripeSpotter('../data/StripeSpotter')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_StripeSpotter.df, metadata_StripeSpotter.root, transform=transform)
dataset_database_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[100:], metadata_StripeSpotter.root, transform=transform)
dataset_query_StripeSpotter = WildlifeDataset(metadata_StripeSpotter.df.iloc[:100], metadata_StripeSpotter.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_StripeSpotter = SIFTFeatures()

query_StripeSpotter, database_StripeSpotter = extractor_StripeSpotter(dataset_query_StripeSpotter), extractor_StripeSpotter(dataset_database_StripeSpotter)

print(f'First 5 query features shape: {[i.shape for i in query_StripeSpotter[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_StripeSpotter[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_StripeSpotter, database_StripeSpotter)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_StripeSpotter = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
predictions_StripeSpotter = classifier_StripeSpotter(sim)

print("Predictions for 100 test Images:-\n",predictions_StripeSpotter)
accuracy_StripeSpotter = np.mean(dataset_query_StripeSpotter.labels_string == predictions_StripeSpotter)
print("Accuracy on StripeSpotter data: {:.2f}%".format(accuracy_StripeSpotter * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_StripeSpotter.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_StripeSpotter.labels_string)

acc = sum(preds == dataset_query_StripeSpotter.labels_string) / len(dataset_query_StripeSpotter.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET StripeSpotter: DOWNLOADING STARTED.


data-20110718.zip: 71.7MB [00:01, 50.4MB/s]                            
data-20110718.z02: 100%|██████████| 78.6M/78.6M [00:01<00:00, 45.9MB/s]
data-20110718.z01: 100%|██████████| 78.6M/78.6M [00:01<00:00, 68.5MB/s]


DATASET StripeSpotter: EXTRACTING STARTED.
DATASET StripeSpotter: FINISHED.



100%|█████████████████████████████████████████████████████████████| 724/724 [01:04<00:00, 11.24it/s]


First 5 query features shape: [(414, 128), (610, 128), (704, 128), (295, 128), (396, 128)]
First 5 database features shape: [(410, 128), (556, 128), (548, 128), (216, 128), (571, 128)]


100%|████████████████████████████████████████████████████████| 72400/72400 [02:29<00:00, 485.11it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[45. 22. 23. ... 11. 31. 39.]
 [29. 26. 26. ... 46. 23. 25.]
 [52. 38. 36. ...  8. 30. 30.]
 ...
 [53. 32. 34. ... 13. 46. 34.]
 [38. 25. 45. ... 12. 48. 40.]
 [45. 38. 43. ... 14. 47. 45.]]
Predictions for 100 test Images:-
 ['09_212' 'S09_161' '04_008' '08_100' '09_200' 'OP05_625' '09_212'
 '02_149' '01_579' '08_034' '01_679' '08_078' '01_460' '10_002' '01_136'
 '09_212' '09_200' 'S09_111' '02_149' 'S09_069' '08_100' '01_363' '01_363'
 '08_078' '08_034' '01_805' '09_355' '08_085' 'OP05_546' '09_212' '08_100'
 'S09_088' '09_182' 'S09_215' '02_162' '01_230' '08_085' '01_661'
 '02_1026' '09_185' '01_136' '01_661' '01_661' '02_161' '01_165' '02_161'
 'S09_174' '02_162' '08_100' 'OP04_225' '02_149' 'S09_067' '09_200'
 '04_008' '01_661' '09_182' '01_805' '02_161' '10_002' '01_036' '02_149'
 '01_165' '09_212' 'S09_211' 'S09_215' '03_045' '01_579' 'OP04_225'
 '08_100' '08_100' '04_008' '01_661' 'OP05_625' 'OP05_625' '02_149'
 

# ***SuperPoint***

In [ ]:
import wildlife_tools.features
help(wildlife_tools.features)

Help on package wildlife_tools.features in wildlife_tools:

NAME
    wildlife_tools.features

PACKAGE CONTENTS
    base
    deep
    memory
    sift
    superpoint

FILE
    /usr/local/lib/python3.10/dist-packages/wildlife_tools/features/__init__.py




In [ ]:
from wildlife_tools.features import  SuperPointFeatures

ImportError: cannot import name 'SuperPointFeatures' from 'wildlife_tools.features' (/usr/local/lib/python3.10/dist-packages/wildlife_tools/features/__init__.py)

In [ ]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT

# Download dataset (if not already downloaded)
datasets.CZoo.get_data('../data/CZoo')

# Load dataset metadata
metadata_CZoo = datasets.CZoo('../data/CZoo')

# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_CZoo.df, metadata_CZoo.root, transform=transform)
dataset_database_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[100:], metadata_CZoo.root, transform=transform)
dataset_query_CZoo = WildlifeDataset(metadata_CZoo.df.iloc[:100], metadata_CZoo.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_CZoo =  SuperPointFeatures()

query_CZoo, database_CZoo = extractor_CZoo(dataset_query_CZoo), extractor_CZoo(dataset_database_CZoo)

print(f'First 5 query features shape: {[i.shape for i in query_CZoo[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_CZoo[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.8])
sim = similarity(query_CZoo, database_CZoo)[0.8]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_CZoo = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
predictions_CZoo = classifier_CZoo(sim)

print("Predictions for 100 test Images:-\n",predictions_CZoo)
accuracy_CZoo = np.mean(dataset_query_CZoo.labels_string == predictions_CZoo)
print("Accuracy on CZoo data: {:.2f}%".format(accuracy_CZoo * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_CZoo.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_CZoo.labels_string)

acc = sum(preds == dataset_query_CZoo.labels_string) / len(dataset_query_CZoo.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET CZoo: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


NameError: name '__file__' is not defined

# ***ArcFaceLoss***

In [ ]:
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=10,
    device='cpu',
    epoch_callback=print_epoch_loss
)

trainer.train()


DATASET IPanda50: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Epoch 0:   0%|                                                               | 0/54 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


# ***TripletLoss***

In [ ]:
import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer, TripletLoss



# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)

# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = ArcFaceLoss(
    num_classes=dataset.num_classes,
    embedding_size=768,
    margin=0.5,
    scale=64
    )

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224',objective=objective, optimizer = optimizer ,num_classes=0, pretrained=True)

DATASET IPanda50: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


In [ ]:
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

In [ ]:
# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(backbone)
query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)

  0%|                                                                         | 0/1 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
100%|███████████████████████████████████████████████████████████████| 53/53 [35:45<00:00, 40.47s/it]


In [ ]:
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)

{'cosine': array([[ 0.02714637,  0.53501487,  0.14056858, ...,  0.38829625,
         0.43481827, -0.07561179],
       [ 0.08373775,  0.5616951 ,  0.03785922, ...,  0.3602667 ,
         0.6493416 , -0.1295084 ],
       [ 0.01190435,  0.4141855 ,  0.23542283, ...,  0.45754877,
         0.5449476 , -0.1335805 ],
       ...,
       [ 0.0320384 ,  0.12872331, -0.14140168, ...,  0.02448   ,
         0.22118227, -0.17661369],
       [-0.08624166,  0.16572046,  0.07199848, ...,  0.14675027,
         0.23473434, -0.02312863],
       [ 0.01022829,  0.06892081,  0.7597883 , ...,  0.4224525 ,
         0.10611828, -0.04714176]], dtype=float32)}


In [ ]:
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on IPanda50 data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)

Predictions for 100 test Images:-
 ['10_hexing' '37_xinger' '01_aoliao' '33_wuyi' '47_yingying' '15_maosun'
 '09_fushun' '10_hexing' '36_xingda' '19_nannan' '08_fulai' '31_shurong'
 '44_yayi' '39_xinghui' '37_xinger' '14_maodou' '14_maodou' '00_aibang'
 '21_nina' '35_xilan' '19_nannan' '33_wuyi' '47_yingying' '49_yuanrun'
 '38_xingfan' '04_chengdui' '16_maotao' '48_yongbang' '11_jiaoao'
 '49_yuanrun' '24_qixi' '10_hexing' '35_xilan' '35_xilan' '02_baolan'
 '31_shurong' '31_shurong' '02_baolan' '23_qiubang' '26_qiyuan'
 '04_chengdui' '35_xilan' '08_fulai' '00_aibang' '10_hexing'
 '29_shuangxiong' '11_jiaoao' '44_yayi' '14_maodou' '30_shuqing'
 '36_xingda' '39_xinghui' '33_wuyi' '10_hexing' '10_hexing' '00_aibang'
 '49_yuanrun' '35_xilan' '04_chengdui' '10_hexing' '39_xinghui' '44_yayi'
 '10_hexing' '36_xingda' '16_maotao' '22_nini' '00_aibang' '39_xinghui'
 '10_hexing' '17_meibang' '00_aibang' '46_yazhu' '14_maodou' '15_maosun'
 '32_susu' '26_qiyuan' '27_rourou' '23_qiubang' '18_miaomia

REAL CODE FORM HeRE

In [ ]:
import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer, TripletLoss



# Download dataset (if not already downloaded)
datasets.IPanda50.get_data('../data/IPanda50')
# Load dataset metadata
metadata = datasets.IPanda50('../data/IPanda50')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224',objective=ArcFaceLoss(), num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=1,
    device='cuda',
    epoch_callback=print_epoch_loss
)



trainer.train()



DATASET IPanda50: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


Epoch 0:   0%|                                                               | 0/54 [00:00<?, ?it/s]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 54/54 [02:06<00:00,  2.05s/it]/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch 0: 100%|██████████████████████████████████████████████████████| 54/54 [02:06<00:00,  2.34s/it]

Epoch 1: Average Loss = 0.2463686764240265


# ***DogFaceNet ***

TripletLoss

In [4]:

# %%
from itertools import chain
import timm
import pandas as pd
import torchvision.transforms as T
from torch.optim import SGD

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.train import ArcFaceLoss, BasicTrainer

import timm
import numpy as np
from wildlife_datasets.datasets import MacaqueFaces
from wildlife_tools.data import WildlifeDataset
import torchvision.transforms as T
from wildlife_datasets import datasets, splits
from wildlife_tools.features import DeepFeatures
from wildlife_tools.similarity import CosineSimilarity
from wildlife_tools.inference import KnnClassifier
from sklearn.metrics import precision_score, recall_score, f1_score

import timm
import itertools
from torch.optim import SGD
from wildlife_tools.train import ArcFaceLoss, BasicTrainer , TripletLoss


# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata = datasets.DogFaceNet('../data/DogFaceNet')
transform = T.Compose([T.Resize([224, 224]), T.ToTensor(), T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))])
dataset = WildlifeDataset(metadata.df, metadata.root, transform=transform)


# Download MegaDescriptor-T backbone from HuggingFace Hub
backbone = timm.create_model('hf-hub:BVRA/MegaDescriptor-T-224', num_classes=0, pretrained=True)

# Arcface loss - needs backbone output size and number of classes.
objective = TripletLoss()

# Optimize parameters in backbone and in objective using single optimizer.
params = itertools.chain(backbone.parameters(), objective.parameters())
optimizer = SGD(params=params, lr=0.001, momentum=0.9)

def print_epoch_loss(trainer, epoch_data):
    # This function will print the average loss at the end of each epoch
    print(f"Epoch {trainer.epoch}: Average Loss = {epoch_data['train_loss_epoch_avg']}")


trainer = BasicTrainer(
    dataset=dataset,
    model=backbone,
    objective=objective,
    optimizer=optimizer,
    epochs=2,
    device='cuda',
    epoch_callback=print_epoch_loss
)

trainer.train()



# # %%
# import torch

# trainer.save("retrained_chks", file_name="tripletloss_IPanda50_retrained_checkpoint.pth")




# %%
dataset_database_P = WildlifeDataset(metadata.df.iloc[100:,:], metadata.root, transform=transform)
dataset_query_P = WildlifeDataset(metadata.df.iloc[:100,:], metadata.root, transform=transform)

# name = 'hf-hub:BVRA/MegaDescriptor-T-224'
extractor_P = DeepFeatures(trainer.model , device = 'cuda')

query_P, database_P = extractor_P(dataset_query_P), extractor_P(dataset_database_P)





# %%
similarity_function = CosineSimilarity()
similarity_P = similarity_function(query_P, database_P)
print(similarity_P)


# %%
classifier_P = KnnClassifier(k=1, database_labels=dataset_database_P.labels_string)
predictions_P = classifier_P(similarity_P['cosine'])
print("Predictions for 100 test Images:-\n",predictions_P)
accuracy_P = np.mean(dataset_query_P.labels_string == predictions_P)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_P * 100))
precision_P = precision_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
recall_P = recall_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
f1_P = f1_score(dataset_query_P.labels_string, predictions_P, average='weighted',zero_division=1)
print("Precision:", precision_P)
print("Recall:", recall_P)
print("F1 Score:", f1_P)




DATASET DogFaceNet: DOWNLOADING STARTED.


DogFaceNet_Dataset_224_1.zip: 75.6MB [00:03, 25.1MB/s]                            


DATASET DogFaceNet: EXTRACTING STARTED.
DATASET DogFaceNet: FINISHED.



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

...

Epoch 1: Average Loss = 0.16545015573501587


Epoch 1: 100%|██████████████████████████████████████████████████████| 66/66 [01:40<00:00,  1.52s/it]


Epoch 2: Average Loss = 0.1578773409128189


100%|███████████████████████████████████████████████████████████████| 65/65 [00:39<00:00,  1.66it/s]

{'cosine': array([[ 0.03972328,  0.20781924,  0.14740594, ...,  0.12605846,
         0.30586195, -0.02830541],
       [-0.0353424 ,  0.22803351,  0.06092985, ...,  0.07184955,
         0.23896131, -0.05035808],
       [ 0.25620338, -0.03554427,  0.09088042, ...,  0.05417535,
         0.05545497,  0.172644  ],
       ...,
       [ 0.4888112 , -0.02861565,  0.35003257, ...,  0.11940275,
        -0.04592966,  0.30399358],
       [-0.05110976,  0.42297965, -0.06133378, ...,  0.14550635,
         0.24097437, -0.13004112],
       [-0.01853296,  0.30375302, -0.08097982, ...,  0.30746043,
         0.27140635, -0.13072297]], dtype=float32)}
Predictions for 100 test Images:-
 ['262' '182' '919' '1122' '428' '378' '1063' '923' '470' '405' '764' '143'
 '635' '1339' '432' '627' '1075' '274' '296' '234' '529' '991' '247' '425'
 '280' '407' '1268' '1248' '446' '538' '739' '448' '1336' '251' '448'
 '459' '563' '210' '345' '1383' '499' '1324' '158' '127' '1398' '1254'
 '1139' '208' '626' '424' '293' '4

DogFaceNet SIFT

In [6]:
import numpy as np
from PIL import Image
import torchvision.transforms as T
from wildlife_datasets import datasets
from wildlife_tools.data import WildlifeDataset
from wildlife_tools.features import SIFTFeatures
import cv2  # Import OpenCV for SIFT



# Download dataset (if not already downloaded)
datasets.DogFaceNet.get_data('../data/DogFaceNet')

# Load dataset metadata
metadata_DogFaceNet = datasets.DogFaceNet('../data/DogFaceNet')


# Define transformations: resize, convert to PIL Image, convert to grayscale, and convert to numpy array
transform = T.Compose([
    T.Resize([224, 224]),  # Resize the image
    # T.ToPILImage(),  # Convert tensor to PIL Image
    T.Grayscale(),  # Convert to grayscale
    # T.ToTensor(),  # Convert PIL Image to numpy array
    # lambda x: (x * 255).astype(np.uint8)  # Convert to 8-bit integer format
])

# Create datasets with transformations
dataset = WildlifeDataset(metadata_DogFaceNet.df, metadata_DogFaceNet.root, transform=transform)
dataset_database_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[100:], metadata_DogFaceNet.root, transform=transform)
dataset_query_DogFaceNet = WildlifeDataset(metadata_DogFaceNet.df.iloc[:100], metadata_DogFaceNet.root, transform=transform)

# Initialize SIFT extractor
sift = cv2.SIFT_create()
extractor_DogFaceNet = SIFTFeatures()

query_DogFaceNet, database_DogFaceNet = extractor_DogFaceNet(dataset_query_DogFaceNet), extractor_DogFaceNet(dataset_database_DogFaceNet)

print(f'First 5 query features shape: {[i.shape for i in query_DogFaceNet[:5]]}')
print(f'First 5 database features shape: {[i.shape for i in database_DogFaceNet[:5]]}')

import timm
import pandas as pd
import torchvision.transforms as T

from wildlife_tools.data import WildlifeDataset, SplitMetadata
from wildlife_tools.features import SIFTFeatures
from wildlife_tools.similarity import MatchDescriptors
from wildlife_tools.inference import KnnClassifier

similarity = MatchDescriptors(descriptor_dim=128, thresholds=[0.7])
sim = similarity(query_DogFaceNet, database_DogFaceNet)[0.7]

print("Number of SIFT correspondences after 0.8 ratio test threshold: \n", sim)


classifier_DogFaceNet = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
predictions_DogFaceNet = classifier_DogFaceNet(sim)

print("Predictions for 100 test Images:-\n",predictions_DogFaceNet)
accuracy_DogFaceNet = np.mean(dataset_query_DogFaceNet.labels_string == predictions_DogFaceNet)
print("Accuracy on DogFaceNet data: {:.2f}%".format(accuracy_DogFaceNet * 100))



# Nearest neigbour classifier using the similarity
classifier = KnnClassifier(k=1, database_labels=dataset_database_DogFaceNet.labels_string)
preds = classifier(sim)
print("Prediction \t", preds)
print("Ground truth \t", dataset_query_DogFaceNet.labels_string)

acc = sum(preds == dataset_query_DogFaceNet.labels_string) / len(dataset_query_DogFaceNet.labels_string)
print('\n Accuracy: ', acc*100,"%")



DATASET DogFaceNet: DOWNLOADING STARTED.
You are trying to download an already downloaded dataset.
        This message may have happened to due interrupted download or extract.
        To force the download use the `force=True` keyword such as
        get_data(..., force=True) or download(..., force=True).
        


100%|███████████████████████████████████████████████████████████| 8263/8263 [04:34<00:00, 30.14it/s]


First 5 query features shape: [(174, 128), (250, 128), (157, 128), (339, 128), (513, 128)]
First 5 database features shape: [(267, 128), (169, 128), (341, 128), (432, 128), (177, 128)]


100%|██████████████████████████████████████████████████████| 826300/826300 [18:06<00:00, 760.54it/s]

Number of SIFT correspondences after 0.8 ratio test threshold: 
 [[19. 14. 13. ... 18. 11. 11.]
 [15. 10. 23. ... 20. 12. 24.]
 [15. 19. 19. ... 13.  8. 17.]
 ...
 [14.  9. 24. ... 18.  7. 18.]
 [ 7. 14. 22. ...  9. 13. 20.]
 [16.  8. 18. ... 18.  9. 19.]]
Predictions for 100 test Images:-
 ['300' '1319' '1296' '223' '428' '1296' '943' '385' '974' '223' '1125'
 '300' '1296' '223' '300' '385' '1296' '385' '1125' '174' '1296' '1296'
 '151' '385' '1125' '1296' '1125' '300' '223' '385' '385' '385' '1296'
 '1296' '274' '96' '1296' '1296' '223' '1296' '751' '300' '300' '127'
 '961' '1296' '1296' '174' '1296' '424' '274' '943' '1296' '300' '1296'
 '300' '1125' '223' '1296' '11' '223' '1296' '96' '814' '1308' '631'
 '1296' '646' '1238' '223' '1296' '1296' '1125' '300' '1169' '458' '223'
 '149' '1296' '622' '167' '1125' '1296' '223' '1296' '1357' '1296' '223'
 '1296' '385' '1296' '943' '1296' '1296' '1125' '1296' '1296' '943' '1020'
 '1296']
Accuracy on DogFaceNet data: 11.00%
Prediction 	 ['30